# Restaurant Recommendation Assist - AI

#### Problem Statement
In today's fast-paced world, people often seek quick and convenient dining options, making restaurants a popular choice. However, with so many restaurants available, customers may find it challenging to discover new options that suit their preferences.
<br> To address this issue, we aim to develop a Restaurant Recommendation System. This system will utilize user preferences, such as cuisine type, location, budget, and dietary restrictions, to recommend the most suitable restaurants in 'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'.


In [1]:
import pandas as pd  # Importing the pandas library for data manipulation
import json
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
restaurant_data = pd.read_csv('restaurant_data_6_states.csv')
restaurant_data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites,features,state,feature_dict,price range
0,AWrROxTbcuP9ADUBCmQx,2019-05-19T17:54:31Z,2019-05-19T17:54:31Z,198 Central Avenue,"Hamburgers and Hot Dogs,American Restaurants,F...",Accommodation & Food Services,White Plains,US,us/ny/whiteplains/198centralavenue/-904778134,41.03715,-73.77877,Nathan's Famous Hot Dogs,10606,NY,https://www.yellowpages.com/white-plains-ny/mi...,http://nathansfamous.com,{'open-hours': 'Do you know the hours for this...,New York,{'open-hours': 'do you know the hours for this...,average
1,AVweEg_oByjofQCxvA1V,2016-07-06T20:11:39Z,2019-05-19T07:41:13Z,527 Bald Eagle Dr,"Caterers,Seafood restaurants,Steak and chop re...",Accommodation & Food Services,Marco Island,US,us/fl/marcoisland/527baldeagledr/-873322027,25.95090,-81.72280,Kretch's Restaurant,34145,FL,https://www.yellowbook.com/profile/kretchs-res...,"http://www.kretchs.com,http://kretchs.com/,htt...",{},Florida,{},average
2,AVwdQloekufWRAb54Asw,2016-05-05T12:40:33Z,2019-05-18T23:45:36Z,3121 Yosemite Blvd,"Asian Restaurants,Caterers,Chinese Restaurants...",Accommodation & Food Services,Modesto,US,us/ca/modesto/3121yosemiteblvd/1235262340,37.63860,-120.93670,Fat Wong Chinese Food To Go,95354,CA,http://www.citysearch.com/profile/1154859/mode...,http://www.fatwongchinesefood.com/,{},California,{},average
3,AWjXLnyyRxPSIh2RASUl,2019-02-10T10:28:04Z,2019-05-18T22:09:17Z,3905 Sw 40th Boulevard,"Fast Food Restaurant,Fast Food",Accommodation & Food Services,Gainesville,US,us/fl/gainesville/3905sw40thboulevard/-2061630068,29.61790,-82.38360,Burger King,32608,FL,https://foursquare.com/v/burger-king/4b4d0620f...,https://locations.bk.com/fl/gainesville/3905-s...,{},Florida,{},average
4,AVwdMBXWByjofQCxmbTu,2016-06-08T16:17:17Z,2019-05-18T21:25:22Z,8256 Orion Ave,"Mexican Restaurant,Restaurant,Mexican Restaura...",Accommodation & Food Services,Van Nuys,US,us/ca/vannuys/8256orionave/-864103396,34.22080,-118.47190,Taco Bell,91406,CA,http://www.citysearch.com/profile/633545994/va...,https://locations.tacobell.com/ca/van-nuys/825...,{'open-hours': 'Regular HoursMon - Thu:9:00 am...,California,{'open-hours': 'regular hoursmon - thu:9:00 am...,below average


In [3]:
restaurant_data.shape

(2681, 20)

In [4]:
restaurant_data['state'].value_counts()

Florida         739
California      727
Texas           426
Washington      422
Pennsylvania    194
New York        173
Name: state, dtype: int64

#### Approach:

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 5 recommendations that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the best restaurant based on the users dietary habits.

## Part 2: System Design


#### Dataset

We have a dataset `restaurant_data.csv` which represents the features of the hotel. The chatbot that we build will leverage LLMs to parse this `features` column and provide recommendations

As you could see in the output above, the chatbot should ask a series of questions to
- Determine the user's requirments. For simplicity, we have used 5 features to encapsulate the user's needs. The 5 features are as follows:
    - Cuisine preference
    - State Preferences
    - Parking
    - Dining in    
    - Budget

- Confirm if the user's requirements have been correctly captured at the end.

After that the chatbot lists down the top recommendations that are the most relevant, and engages in further conversation to help the user find the best one.


#### Building the Chatbot

Now let's go ahead and understand the system design for the chatbot.

`Stage 1`

- Intent Clarity Layer
- Intent Confirmation Layer

`Stage 2`

- Product Mapping Layer
- Product Information Extraction Layer

`Stage 3`

- Product Recommendation Layer

## Implementation

## Stage 1

In [5]:
# Read the OpenAI API key
# mention the text file where the open ai api key is mentioned
# file_path = "OPENAI_API_Key.txt"
file_path = input("Enter the file path for OpenAI API key")

if os.path.exists(file_path):
    openai.api_key = open(file_path, "r").read().strip()
else:
    api_key = input("File does not exist. Please enter the open AI API key")
    openai.api_key = api_key
    
# Read the OpenAI API key
os.environ['OPENAI_API_KEY'] = openai.api_key

Enter the file path for OpenAI API key
File does not exist. Please enter the open AI API keysk-proj-fA8tYn1GVaWDXQrva6vsT3BlbkFJBEqkbGEea1Pobi4QnFJt


### Implementing Intent Clarity and Intent Confirmation Layers

Let's start with the first part of the implementation - building the `intent clarity` and `intent confirmation` layers. As mentioned earlier, this layer helps in identifying the user requirements and passing it on to the product matching layer. 

In [6]:
exploded_values = restaurant_data['categories'].apply(lambda x: x.split(',')).explode()
# Get unique values
unique_values = exploded_values.unique()
unique_values

array(['Hamburgers and Hot Dogs', 'American Restaurants',
       'Fast Food Restaurants', 'Restaurants', 'Caterers',
       'Seafood restaurants', 'Steak and chop restaurants',
       'Crabs and crab houses', 'Restaurant Delivery Service',
       'American restaurants', 'Seafood', 'Karaoke',
       'Take Out Restaurants', 'Family Style Restaurants', 'Restaurant',
       'Steak Houses', 'Night Clubs', 'Bars', 'Asian Restaurants',
       'Chinese Restaurants', 'Fast Food Restaurant', 'Fast Food',
       'Mexican Restaurant', 'Mexican Restaurants', 'Mexican',
       'Breakfast', 'Vegetarian', 'Cafes', 'Hamburgers', 'Chicken',
       'Italian Restaurant', 'Italian', 'Hot dog restaurants',
       'Hamburger restaurants', 'Italian Restaurants', 'Pizza',
       'Mediterranean Restaurants', 'Sandwich Shops',
       'Traditional American', ' Mexican Restaurant', ' and Taco Place',
       'Taco Place', 'wich Place', 'Catering', 'Sandwich Place',
       'Delis And Delicatessens', 'Melbourne Resta

In [7]:
restaurant_data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites,features,state,feature_dict,price range
0,AWrROxTbcuP9ADUBCmQx,2019-05-19T17:54:31Z,2019-05-19T17:54:31Z,198 Central Avenue,"Hamburgers and Hot Dogs,American Restaurants,F...",Accommodation & Food Services,White Plains,US,us/ny/whiteplains/198centralavenue/-904778134,41.03715,-73.77877,Nathan's Famous Hot Dogs,10606,NY,https://www.yellowpages.com/white-plains-ny/mi...,http://nathansfamous.com,{'open-hours': 'Do you know the hours for this...,New York,{'open-hours': 'do you know the hours for this...,average
1,AVweEg_oByjofQCxvA1V,2016-07-06T20:11:39Z,2019-05-19T07:41:13Z,527 Bald Eagle Dr,"Caterers,Seafood restaurants,Steak and chop re...",Accommodation & Food Services,Marco Island,US,us/fl/marcoisland/527baldeagledr/-873322027,25.95090,-81.72280,Kretch's Restaurant,34145,FL,https://www.yellowbook.com/profile/kretchs-res...,"http://www.kretchs.com,http://kretchs.com/,htt...",{},Florida,{},average
2,AVwdQloekufWRAb54Asw,2016-05-05T12:40:33Z,2019-05-18T23:45:36Z,3121 Yosemite Blvd,"Asian Restaurants,Caterers,Chinese Restaurants...",Accommodation & Food Services,Modesto,US,us/ca/modesto/3121yosemiteblvd/1235262340,37.63860,-120.93670,Fat Wong Chinese Food To Go,95354,CA,http://www.citysearch.com/profile/1154859/mode...,http://www.fatwongchinesefood.com/,{},California,{},average
3,AWjXLnyyRxPSIh2RASUl,2019-02-10T10:28:04Z,2019-05-18T22:09:17Z,3905 Sw 40th Boulevard,"Fast Food Restaurant,Fast Food",Accommodation & Food Services,Gainesville,US,us/fl/gainesville/3905sw40thboulevard/-2061630068,29.61790,-82.38360,Burger King,32608,FL,https://foursquare.com/v/burger-king/4b4d0620f...,https://locations.bk.com/fl/gainesville/3905-s...,{},Florida,{},average
4,AVwdMBXWByjofQCxmbTu,2016-06-08T16:17:17Z,2019-05-18T21:25:22Z,8256 Orion Ave,"Mexican Restaurant,Restaurant,Mexican Restaura...",Accommodation & Food Services,Van Nuys,US,us/ca/vannuys/8256orionave/-864103396,34.22080,-118.47190,Taco Bell,91406,CA,http://www.citysearch.com/profile/633545994/va...,https://locations.tacobell.com/ca/van-nuys/825...,{'open-hours': 'Regular HoursMon - Thu:9:00 am...,California,{'open-hours': 'regular hoursmon - thu:9:00 am...,below average


In [8]:
restaurant_data['price range'].unique()

array(['average', 'below average', 'above average'], dtype=object)

In [9]:
restaurant_data['feature_dict'] = restaurant_data['features'].apply(lambda x:ast.literal_eval(str(x).lower()))

In [10]:
# Define the key you want to find the value for
key_to_find = 'categories'
# Extract the value associated with the key
restaurant_data['feature_dict'].apply(lambda x: x.get(key_to_find, None)).value_counts()


fast food restaurants, restaurants, sandwich shops, take out restaurants                                                                                                                                                                                                                                                            111
fast food restaurants, hamburgers & hot dogs, restaurants, take out restaurants                                                                                                                                                                                                                                                      77
fast food restaurants, hamburgers & hot dogs, restaurants                                                                                                                                                                                                                                                                            54
pizza, fast food

In [11]:
example_user_dict = {'cuisine':'values',
                     'delivery location':'values',
                     'is free parking':'values',
                     'is dine-in':'values',
                     'price range':'values'}

example_user_dict.keys()

dict_keys(['cuisine', 'delivery location', 'is free parking', 'is dine-in', 'price range'])

In [69]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'cuisine':'values',
                         'delivery location':'values',
                         'is free parking':'values',
                         'is dine-in':'values',
                         'price range':'values'}

    example_user_req = {'cuisine': "_",
                        'delivery location': "_",
                        'is free parking': "_",
                        'is dine-in': "_",
                        'price range': "_"}

    system_message = f"""
    You are an excellent food blogger or food critic expert and your goal is to recommend the best restaurant for a user in
    'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'.
    You need to ask relevant questions and understand the user food and dietary preferences by analysing the user's responses.
    You final objective is to fill the values for the different keys ('cuisine','delivery location','is free parking','is dine-in','price range') in the python dictionary and be confident of the values.
    These key value pairs define the user's food and dietary preferences.
    The python dictionary looks like this
    {{'cuisine':'values','delivery location':'values','is free parking':'values','is dine-in':'values','price range':'values'}}
    The value for 'delivery location' should be either 'New York' or 'NY', 'Florida' or 'FL', 'California' or 'CA', 'Pennsylvania' or 'PA', 'Texas' or 'TX','Washington' or 'WA' extracted from the user's response.
    The value for 'is free parking','is dine-in' should be either 'Yes' or 'No'.
    The value for 'price range' should be either 'average', 'below average', 'above average'.
    The value for 'cuisine' can be any kind of restaurants like 'mexican', 'chinese', 'indian', 'japanese', 'chinese', 'asian', 'fast food', 'pizza', 'hamburgers', 'hot dogs', 'sea food' etc

    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's food and dietary preferences. \n
    If their food and cuisine preference is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'cuisine', 'delivery location', is free parking','is dine-in','price range'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: Hi there! I'm looking for a good restaurant to have dinner tonight. Any recommendations?

    Assistant: Of course! Which US state or province is your ideal location?

    User: California or Pennsylvania

    Assistant: Noted! Could you please tell what type of cuisine are you in the mood for?

    User: I'm open to anything, but I'm leaning towards Italian or Mexican food.

    Assistant: Thank you for providing that information. Would you require parking facilities?

    User: Yes. I need free parking.
    .
    Assistant: I appreciate you sharing this information. Which do you prefer: takeout or dining in?

    User: I prefer free dining in.

    Assistant: Could you kindly let me know your budget? This will help me find options that fit within your price range.

    User: Less than 25 dollar

    Assistant: Great! Then For Italian cuisine, I recommend trying out "Tonelli's Takeout" in '435 Germantown Pike, Pennsylvania'. They have amazing Pizza and a cozy atmosphere. For Mexican food, "Taco Bell" in "8256 Orion Ave, California" is fantastic. They serve authentic Mexican cuisine.

    User: Thank you so much for your help!

    Assistant: {example_user_dict}
               {delimiter}

    Start with a short welcome message and encourage the user to share their requirements. Ask user to type 'Exit' to end the chat.
    Keep in mind while asking for delivery location always ask to choose from one of the following states in US 
    'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [13]:
restaurant_data.head(10)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites,features,state,feature_dict,price range
0,AWrROxTbcuP9ADUBCmQx,2019-05-19T17:54:31Z,2019-05-19T17:54:31Z,198 Central Avenue,"Hamburgers and Hot Dogs,American Restaurants,F...",Accommodation & Food Services,White Plains,US,us/ny/whiteplains/198centralavenue/-904778134,41.03715,-73.77877,Nathan's Famous Hot Dogs,10606,NY,https://www.yellowpages.com/white-plains-ny/mi...,http://nathansfamous.com,{'open-hours': 'Do you know the hours for this...,New York,{'open-hours': 'do you know the hours for this...,average
1,AVweEg_oByjofQCxvA1V,2016-07-06T20:11:39Z,2019-05-19T07:41:13Z,527 Bald Eagle Dr,"Caterers,Seafood restaurants,Steak and chop re...",Accommodation & Food Services,Marco Island,US,us/fl/marcoisland/527baldeagledr/-873322027,25.95090,-81.72280,Kretch's Restaurant,34145,FL,https://www.yellowbook.com/profile/kretchs-res...,"http://www.kretchs.com,http://kretchs.com/,htt...",{},Florida,{},average
2,AVwdQloekufWRAb54Asw,2016-05-05T12:40:33Z,2019-05-18T23:45:36Z,3121 Yosemite Blvd,"Asian Restaurants,Caterers,Chinese Restaurants...",Accommodation & Food Services,Modesto,US,us/ca/modesto/3121yosemiteblvd/1235262340,37.63860,-120.93670,Fat Wong Chinese Food To Go,95354,CA,http://www.citysearch.com/profile/1154859/mode...,http://www.fatwongchinesefood.com/,{},California,{},average
3,AWjXLnyyRxPSIh2RASUl,2019-02-10T10:28:04Z,2019-05-18T22:09:17Z,3905 Sw 40th Boulevard,"Fast Food Restaurant,Fast Food",Accommodation & Food Services,Gainesville,US,us/fl/gainesville/3905sw40thboulevard/-2061630068,29.61790,-82.38360,Burger King,32608,FL,https://foursquare.com/v/burger-king/4b4d0620f...,https://locations.bk.com/fl/gainesville/3905-s...,{},Florida,{},average
4,AVwdMBXWByjofQCxmbTu,2016-06-08T16:17:17Z,2019-05-18T21:25:22Z,8256 Orion Ave,"Mexican Restaurant,Restaurant,Mexican Restaura...",Accommodation & Food Services,Van Nuys,US,us/ca/vannuys/8256orionave/-864103396,34.22080,-118.47190,Taco Bell,91406,CA,http://www.citysearch.com/profile/633545994/va...,https://locations.tacobell.com/ca/van-nuys/825...,{'open-hours': 'Regular HoursMon - Thu:9:00 am...,California,{'open-hours': 'regular hoursmon - thu:9:00 am...,below average
5,AVwdTP_RkufWRAb54dbQ,2015-11-23T19:00:21Z,2019-05-18T20:32:24Z,2994 Crawfordville Hwy,"Restaurant,Fast Food Restaurant,Fast Food,Hamb...",Accommodation & Food Services,Crawfordville,US,us/fl/crawfordville/2994crawfordvillehwy/69649...,30.17840,-84.37560,Hardee's,32327,FL,http://www.citysearch.com/profile/2284945/craw...,https://www.hardees.com/?utm_source=Yext&utm_m...,{},Florida,{},average
6,AWiEBGy03-Khe5l_ym4U,2019-01-25T07:20:13Z,2019-05-18T20:32:15Z,3750 NW 13th Street,"Fast Food,Chicken,Restaurants",Accommodation & Food Services,Gainesville,US,us/fl/gainesville/3750nw13thstreet/106088,29.68580,-82.33970,KFC,32609,FL,http://www.citysearch.com/profile/44602712/gai...,https://www.kfc.com/store-locator?query=32609,{},Florida,{},average
7,AWmf_r7xIxWefVJwMY1g,2019-03-21T11:03:37Z,2019-05-18T17:50:19Z,990 N. Western Ave.,"Mexican,Breakfast,Vegetarian,Fast Food,Restaur...",Accommodation & Food Services,Los Angeles,US,us/ca/losangeles/990n.westernave./-864103396,34.08830,-118.30880,Taco Bell,90029,CA,http://www.citysearch.com/profile/56462/los_an...,https://locations.tacobell.com/ca/los-angeles/...,{},California,{},average
8,AVwc0GNEByjofQCxieao,2016-03-29T04:46:55Z,2019-05-18T16:24:17Z,435 Germantown Pike,"Italian Restaurant,Caterers,Italian,Hot dog re...",Accommodation & Food Services,Lafayette Hill,US,us/pa/lafayettehill/435germantownpike/697681611,40.09140,-75.25930,Tonelli's Takeout,19444-1815,PA,https://www.yellowpages.com/lafayette-hill-pa/...,"http://www.tonellistakeout.com/,http://www.you...",{'open-hours': 'Regular HoursMon - Thu:11:00 a...,Pennsylvania,{'open-hours': 'regular hoursmon - thu:11:00 a...,average
9,AWmfMlKga4HuVbedosmu,2019-03-21T07:08:33Z,2019-05-18T16

In [14]:
debug_conversation = initialize_conversation()
# Let's look at the content in the debug_conversation key
print(debug_conversation[0]['content'])


    You are an excellent food blogger or food critic expert and your goal is to recommend the best restaurant for a user in
    'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'.
    You need to ask relevant questions and understand the user food and dietary preferences by analysing the user's responses.
    You final objective is to fill the values for the different keys ('cuisine','delivery location','is free parking','is dine-in','price range') in the python dictionary and be confident of the values.
    These key value pairs define the user's food and dietary preferences.
    The python dictionary looks like this
    {'cuisine':'values','delivery location':'values','is free parking':'values','is dine-in':'values','price range':'values'}
    The value for 'delivery location' should be either 'New York' or 'NY', 'Florida' or 'FL', 'California' or 'CA', 'Pennsylvania' or 'PA', 'Texas' or 'TX','Washington' or 'WA' extracted from the user's response.
    The 

In [15]:
# Let's initialise conversation
system_message = initialize_conversation()
print(system_message[0]["content"])


    You are an excellent food blogger or food critic expert and your goal is to recommend the best restaurant for a user in
    'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'.
    You need to ask relevant questions and understand the user food and dietary preferences by analysing the user's responses.
    You final objective is to fill the values for the different keys ('cuisine','delivery location','is free parking','is dine-in','price range') in the python dictionary and be confident of the values.
    These key value pairs define the user's food and dietary preferences.
    The python dictionary looks like this
    {'cuisine':'values','delivery location':'values','is free parking':'values','is dine-in':'values','price range':'values'}
    The value for 'delivery location' should be either 'New York' or 'NY', 'Florida' or 'FL', 'California' or 'CA', 'Pennsylvania' or 'PA', 'Texas' or 'TX','Washington' or 'WA' extracted from the user's response.
    The 

In [16]:
# This function perform LLM call using the Chat Completions API to get the LLM response.
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [17]:
# We've created a small helper test function to ensure the model's response is consistent.
def iterate_llm_response(funct, debug_response, num = 10):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

Let's pass the initialized conversation `debug_conversation` and see what is the assistant's response.

In [18]:
debug_user_input = "Hi I'm Krishna! I'm looking for a good restaurant to have dinner."

In [19]:
debug_conversation.append({"role": "user", "content": debug_user_input})
# print(debug_conversation[0]["content"]) # System Message
print(debug_conversation[1]["content"]) # User Input

Hi I'm Krishna! I'm looking for a good restaurant to have dinner.


In [20]:
# Getting the response from the Assistant by passing the conversation to the Chat Completions API
debug_response_assistant = get_chat_completions(debug_conversation)
display(debug_response_assistant)

"Hello Krishna! That's great to hear. To assist you better, could you please let me know in which US state you would like to have dinner tonight? You can choose from 'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'."

Let's play around a bit and add the following user's input `debug_user_input` to the conversation `debug_conversation` and see what the assistant responds with.

In [21]:
# Let's append this to the conversation list
debug_conversation.append(({"role": "system", "content": debug_response_assistant}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an excellent food blogger or food critic expert and your goal is to recommend the best restaurant for a user in\n    \'New York\', \'Florida\', \'California\', \'Pennsylvania\', \'Texas\' or \'Washington\'.\n    You need to ask relevant questions and understand the user food and dietary preferences by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'cuisine\',\'delivery location\',\'is free parking\',\'is dine-in\',\'price range\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s food and dietary preferences.\n    The python dictionary looks like this\n    {\'cuisine\':\'values\',\'delivery location\':\'values\',\'is free parking\':\'values\',\'is dine-in\':\'values\',\'price range\':\'values\'}\n    The value for \'delivery location\' should be either \'New York\' or \'NY\', \'Florida\' or \'FL\', \'California\' or \'CA\', \'Pen

Typically, whenever the chatbot is interacting with the user, all the conversations should be moderated to identify any inappropriate content. Let's look at the function that can help with it.

### `moderation_check()`:
 This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [22]:
# Define a function called moderation_check that takes user_input as a parameter.
# This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [23]:
moderation_check(debug_response_assistant)

'Not Flagged'

### `intent_confirmation_layer()`:

This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
   - 'cuisine'
   - 'delivery location'
   - 'is free parking'
   - 'is dine-in'
   - 'price range'

In [24]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user's food and dietary preferences captured through 7 keys.
    You are provided an input. 
    You need to evaluate if the input text has the following keys 
    {{'cuisine':'values',
     'delivery location':'values',
     'is free parking':'values',
     'is dine-in':'values',
     'price range':'values'}}
     
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

Let's apply the function to the assistant's reponse and see if it has captured the user profile.

In [25]:
debug_response_assistant

"Hello Krishna! That's great to hear. To assist you better, could you please let me know in which US state you would like to have dinner tonight? You can choose from 'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'."

In [26]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
display(debug_confirmation)

{'result': 'No', 'reason': "Key 'cuisine' is missing"}

In [27]:
# Printing the value for better clarity
print("Result:",debug_confirmation.get('result'),"\t", "Reason:", debug_confirmation.get('reason'))

Result: No 	 Reason: Key 'cuisine' is missing


Now, you can keep adding user and assistant responses to debug_conversation and get to a point where intent_confirmation_layer() gives yes as a response. Let's see if the following response by the assistant passes the intent_confirmation_layer() test.

In [28]:
#Let's add the above assistant response to the debug_conversation.
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})

Let's say that after a series of conversations you get the following response from the assistant.

In [29]:
# Example 1 - Let's check with the confirmation_layer if all the keys are present
debug_response_assistant_1 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is your dietary and food preference:
{{'cuisine':'Mexican',
    'food':'burger',
    'delivery location':'TX',
    'is free parking':'No',
    'is dine-in':'Yes',
    'price range':'average'}}
"""
#Note that you are using double curly braces

print(debug_response_assistant_1)


Great, thank you for clarifying your requirements.
Based on your inputs, here is your dietary and food preference:
{'cuisine':'Mexican',
    'food':'burger',
    'delivery location':'TX',
    'is free parking':'No',
    'is dine-in':'Yes',
    'price range':'average'}



Do you think it'll pass the `intent_confirmation_layer()` test?

 Let's try it out.

In [30]:
response = intent_confirmation_layer(debug_response_assistant_1)
response.get('result') # Extract the result key from the dictionary

'No'

In [31]:
# Example 2 - Let's check confirmation_layer if all the keys are present
debug_response_assistant_2 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is your dietary and food preference:
{{'cuisine':'Mexican',
    'is free parking':'No',
    'is dine-in':'Yes',
    'price range':'average'}}
"""
#Note that you are using double curly braces

print(debug_response_assistant_2)


Great, thank you for clarifying your requirements.
Based on your inputs, here is your dietary and food preference:
{'cuisine':'Mexican',
    'is free parking':'No',
    'is dine-in':'Yes',
    'price range':'average'}



In [32]:
intent_confirmation_layer(debug_response_assistant_2)
# iterate_llm_response(intent_confirmation_layer, debug_response_assistant_2)

{'result': 'No', 'reason': 'Missing keys: delivery location'}

In [33]:
# Example 3 - Let's check confirmation_layer if all the keys are present
debug_response_assistant_3 = f"""
Great, thank you for clarifying your requirements.
Based on your inputs, here is your dietary and food preference:
{{  'cuisine':'burger',
    'delivery location':'TX',
    'is free parking':'No',
    'is dine-in':'Yes',
    'drive through':'No',
    'price range':'average'}}
"""
#Note that you are using double curly braces

print(debug_response_assistant_3)


Great, thank you for clarifying your requirements.
Based on your inputs, here is your dietary and food preference:
{  'cuisine':'burger',
    'delivery location':'TX',
    'is free parking':'No',
    'is dine-in':'Yes',
    'drive through':'No',
    'price range':'average'}



In [34]:
intent_confirmation_layer(debug_response_assistant_3)
# iterate_llm_response(intent_confirmation_layer, debug_response_assistant_3)

{'result': 'No',
 'reason': "Incorrect keys provided. Missing 'drive through' key and 'price range' key is incorrect"}

Let's now look at the working of `dictionary_present()`.

### `dictionary_present()`:

This function checks whether the response from the chatbot is a Python dictionary or not. 

In [35]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'cuisine':'burger',
                'delivery location':'TX',
                'is free parking':'No',
                'is dine-in':'Yes',
                'price range':'average'}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
                'cuisine': any cuisines like mexican, indian, burger etc.,
                'delivery location': any state (or it's abbreviation) in US,
                'is free parking': 'Yes / No',
                'is dine-in':'Yes / No',
                'price range': 'average / below average / above average'
            }}
            
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - 
                'cuisine':'burger',
                'delivery location':'TX',
                'is free parking':'No',
                'is dine-in':'Yes',
                'price range':'average'
                
            output 1: {{
                'cuisine':'burger',
                'delivery location':'TX',
                'is free parking':'No',
                'is dine-in':'Yes',
                'price range':'average'
            }}
                
            input 2:    'cuisine':'Mexican',
                        'delivery location':'California',
                        'is free parking':'Yes',
                        'is dine-in':'No',
                        'price range':'below average'
                        
            output 2: {{'cuisine':'Mexican',
                        'delivery location':'California',
                        'is free parking':'Yes',
                        'is dine-in':'No',
                        'price range':'below average'}}

            input 3: Here is your user dietray and food preferences 'cuisine':'Italian','delivery location':'Florida','is free parking':'No','is dine-in':'Yes','price range':'above average'
            output 3: {{'cuisine':'Italian','delivery location':'Florida','is free parking':'No','is dine-in':'Yes','price range':'above average'}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

Let's start by passing the `debug_response_assistant`.

In [36]:
debug_response_assistant_n = """
{'cuisine':'burger',
                'delivery location':'TX',
                'is free parking':'No',
                'is dine-in':'Yes',
                'price range':'average'}
"""

In [37]:
response_dict_n = dictionary_present(debug_response_assistant_n)
display(response_dict_n)

{'cuisine': 'burger',
 'delivery location': 'TX',
 'is free parking': 'No',
 'is dine-in': 'Yes',
 'price range': 'average'}

In [38]:
type(response_dict_n)

dict

What if you pass something like this where it is not in the form of a dictionary? Or some key or some values are missing? Let's see.

In [39]:
debug_response_assistant_n = f"""Thank you for the information
Based on your preference, I will consider this while recommending suitable restaurants for you.
Here is the final recommendation :
- 'cuisine': 'burger'
- 'delivery location': 'TX',
- 'is free parking': 'No',
- 'is dine-in': 'Yes',
- 'price range': 'average'

Please note that these specifications are based on your dietary preference.
Let me know if there's anything else I can assist you with!"""

In [40]:
response_dict_n = dictionary_present(debug_response_assistant_n)
display(response_dict_n)

{'cuisine': 'burger',
 'delivery location': 'TX',
 'is free parking': 'No',
 'is dine-in': 'Yes',
 'price range': 'average'}

In [41]:
type(response_dict_n)

dict

In [42]:
# Check for LLM function's consistency
iterate_llm_response(dictionary_present, debug_response_assistant_n)

Iteration: 0
{'cuisine': 'burger', 'delivery location': 'TX', 'is free parking': 'No', 'is dine-in': 'Yes', 'price range': 'average'}
--------------------------------------------------
Iteration: 1
{'cuisine': 'burger', 'delivery location': 'TX', 'is free parking': 'No', 'is dine-in': 'Yes', 'price range': 'average'}
--------------------------------------------------
Iteration: 2
{'cuisine': 'burger', 'delivery location': 'TX', 'is free parking': 'No', 'is dine-in': 'Yes', 'price range': 'average'}
--------------------------------------------------
Iteration: 3
{'cuisine': 'burger', 'delivery location': 'TX', 'is free parking': 'No', 'is dine-in': 'Yes', 'price range': 'average'}
--------------------------------------------------
Iteration: 4
{'cuisine': 'burger', 'delivery location': 'TX', 'is free parking': 'No', 'is dine-in': 'Yes', 'price range': 'average'}
--------------------------------------------------
Iteration: 5
{'cuisine': 'burger', 'delivery location': 'TX', 'is free park

Let's quickly take a look at the code that we have run until now.

In [43]:
debug_conversation = initialize_conversation()
debug_user_input = "Hi I'm Krishna! I'm looking for a good restaurant to have dinner."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
# After a series of conversation...
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

{'cuisine': 'burger', 'delivery location': 'TX', 'is free parking': 'No', 'is dine-in': 'Yes', 'price range': 'average'}


So, now that you have the user requirement stored in `response_dict_n`. No we will find the services offered by restaurants and map them to a dictionary.

## Stage 2

### Implementing the Product Mapping and Information Extraction Layers

In this section, we take in the output of the previous layers, i.e. the user's requirement, which is in the format of a Python dictionary.
<br>
Next we will provide the top restaurant recommendations based on user's requirement.

This stage consists of the following helper functions that will implement the information extraction and product matching layers.

In [44]:
def product_map_layer(dietary_description):
    delimiter = "#####"

    dietary_spec = {
        'cuisine':'(Type of Cuisine or Food)',
        'delivery location':'(any state or its abbreviation in US)',
        'is free parking':'(Yes or No)',
        'is dine-in':'(Yes or No)',
        'price range':'(average / below average / above average)'
    }

    prompt=f"""
    You are a User Dietary Preference Classifier whose objective is to extract key features and classify them based on user's food and dietary preferences.
    To analyze each dietary preference, perform the following steps:
    Step 1: Extract the User Dietary or Food Preference  features from the description {dietary_description}
    Step 2: Store the extracted features in {dietary_spec} \
    Step 3: Classify each of the items in {dietary_spec} into values based on the following rules: \
    {delimiter}
    
    cuisine:
    - Any type of Cuisine or Food \n

    delivery location:
    - Any state or its abbreviation in US \n
    
    is free parking:
    - Yes: <<< if 'is free parking' value mentioned is 'lot', 'private','free','garage', 'yes', 'true' etc>>> , \n
    - No: <<< if 'is free parking' value mentioned is 'paid', 'street','self', 'no', 'false' etc >>> , \n

    is dine-in:
    - Yes: <<< if 'is dine-in' value mentioned is 'dine-in', 'dining in','reserve a seat', 'yes', 'true' etc>>> , \n
    - No: <<< if 'is dine-in' value mentioned is 'take away', 'delivery', 'take out','no', 'false'  etc >>> , \n
    
    price range:
    - average: <<< if value is numeric and is greater than 10 dollar and less than 20 dollar or if value is non numeric and mentioned as average or normal>>> , \n
    - below average: <<< if value is numeric and is less than 10 dollar or if value is non numeric and mentioned as below average >>> , \n
    - above average: <<< if value is numeric and is greater than 20 dollar or if value is non numeric and mentioned as above average or premium or expensive or luxury >>> , \n
    
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    
    input 1: "{{ 'categories': 'Hamburgers and Hot Dogs,American Restaurants,Fast Food Restaurants,Restaurants',
                'name': "Nathan's Famous Hot Dogs",
                'state': 'New York',
                'feature_dict': {{'open-hours': 'do you know the hours for this business?',
                'extra-phones': 'phone:(914) 497-5421fax:(914) 682-2784',
                'location-description': 'county ctr',
                'aka': 'westchester county center',
                'weblinks': 'http://www.nathansfamous.com',
                'categories': 'hamburgers & hot dogs, american restaurants, fast food restaurants, restaurants ',
                'cuisines': 'burgers, american, hot dogs, fast food'}},
                'price range': 'average'}}" 
    output 1: {{
        'cuisine': 'Hamburgers and Hot Dogs,American Restaurants,Fast Food Restaurants,Restaurants',
        'delivery location': 'New York',
        'is free parking':'Yes',
        'is dine-in':'Yes',
        'price range':'average'
    }}
    
    input 2: "{{  'categories': 'Caterers,Seafood restaurants,Steak and chop restaurants,Crabs and crab houses,Restaurant Delivery Service,American restaurants,Seafood,Karaoke,Fast Food Restaurants,Take Out Restaurants,Family Style Restaurants,Restaurants,Restaurant,Steak Houses,Night Clubs,Bars',
                 'name': "Kretch's Restaurant",
                 'state': 'Florida',
                 'feature_dict': {{}},
                 'price range': 'average'}}"
    output 2: {{'cuisine': 'Caterers,Seafood restaurants,Steak and chop restaurants,Crabs and crab houses,Restaurant Delivery Service,American restaurants,Seafood,Karaoke,Fast Food Restaurants,Take Out Restaurants,Family Style Restaurants,Restaurants,Restaurant,Steak Houses,Night Clubs,Bars',
                'delivery location': 'Florida',
                'is free parking':'Yes',
                'is dine-in':'Yes',
                'price range':'average'}}
    
    input 3: "{{  'categories': 'Mexican Restaurant,Restaurant,Mexican Restaurants,Mexican,Breakfast,Vegetarian,Fast Food Restaurant,Fast Food Restaurants,Fast Food,Cafes,Restaurants',
                 'name': 'Taco Bell',
                 'state': 'California',
                 'feature_dict':{{'open-hours': 'regular hoursmon - thu:9:00 am - 1:00 amfri - sat:9:00 am - 2:00 amsun:9:00 am - 1:00 am',
                  'brands': 'airheads, brisk, cinnabon, dr pepper, lipton, mtn dew, mountain dew, pepsi, sierra mist, starburst',
                  'payment': 'master card, visa, amex, cash, discover',
                  'neighborhoods': 'north hills east, north valley',
                  'weblinks': 'https://locations.tacobell.com/ca/van-nuys/8256-orion-ave.htmlhttps://www.tacobell.com/foodhttps://www.tacobell.com/food',
                  'categories': 'fast food restaurants, mexican restaurants, restaurants ',
                  'wi-fi': 'no',
                  'bike parking': 'yes',
                  'attire': 'casual',
                  'drive through': 'yes',
                  'good for groups': 'yes',
                  'good for kids': 'yes',
                  'take out': 'yes',
                  'cuisines': 'mexican, fast food, latin american, vegetarian, omelets',
                  'price range ': 'below average',
                  'outdoor seating': 'no',
                  'takes reservations': 'no'}},
                 'price range': 'below average'}}"
                 
    output 3: {{'cuisine': 'Mexican Restaurant,Restaurant,Mexican Restaurants,Mexican,Breakfast,Vegetarian,Fast Food Restaurant,Fast Food Restaurants,Fast Food,Cafes,Restaurants',
                'delivery location': 'California',
                'is free parking':'yes',
                'is dine-in':'No',
                'price range':'below average'}}
    
    input 4: "{{  'categories': 'Italian Restaurant,Caterers,Italian,Hot dog restaurants,Hamburger restaurants,Take Out Restaurants,Fast Food Restaurants,Restaurants,Italian Restaurants,Pizza,American Restaurants,Mediterranean Restaurants,Hamburgers and Hot Dogs,Sandwich Shops',
                 'name': "Tonelli's Takeout",
                 'state': 'Pennsylvania',
                 'feature_dict': {{'open-hours': 'regular hoursmon - thu:11:00 am - 9:00 pmfri - sat:11:00 am - 10:00 pmsun:12:00 pm - 12:00 am',
                  'extra-phones': 'phone:(610) 825-1188fax:(610) 825-1188',
                  'payment': 'amex, discover, master card, visa, all major credit cards',
                  'weblinks': 'https://pizzamansprinter.com/tonellistakeout.pdf',
                  'categories': 'pizza, caterers, italian restaurants, restaurants, sandwich shops ',
                  'parking': 'lot, private',
                  'wi-fi': 'no',
                  'attire': 'casual',
                  'cuisines': 'caterers, sandwiches, italian, european, continental, pizza',
                  'alcohol': 'no',
                  'takes reservations': 'no',
                  'outdoor seating': 'no',
                  'price range ': 'average'}},
                 'price range': 'average'}}"
    output 4: {{'cuisine': 'Italian Restaurant,Caterers,Italian,Hot dog restaurants,Hamburger restaurants,Take Out Restaurants,Fast Food Restaurants,Restaurants,Italian Restaurants,Pizza,American Restaurants,Mediterranean Restaurants,Hamburgers and Hot Dogs,Sandwich Shops',
                'delivery location': 'Pennsylvania',
                'is free parking':'Yes',
                'is dine-in':'Yes',
                'price range':'average'}}
    
    
    {delimiter}
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {dietary_spec} for the following dietary preference {dietary_description}."""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format = True)

    return response

In [45]:
def dietary_preference_mapping(row_data):
    output = {}
    cuisines = []
    delivery_location = ""
    is_free_parking = "no"
    is_dine_in = "yes"
    drive_thru = "no"
    price_range = "average"
    
    if 'categories' in row_data:
        cuisines = row_data['categories'].split(',')
    if 'feature_dict' in row_data:
        feature_dict = row_data['feature_dict']
        if 'categories' in feature_dict:
            cuisines.extend(feature_dict['categories'].split(','))
            
        if 'parking' in feature_dict:
            is_free_parking = 'yes' if any(item in ['lot', 'private','free','garage', 'yes', 'true'] for item in feature_dict['parking'].split(',')) else 'no'
        elif 'bike parking' in feature_dict:
            is_free_parking = feature_dict['bike parking']
            
    
    if 'state' in row_data:
        delivery_location = row_data['state']

    if 'price range' in row_data:
        price_range = row_data['price range']
        
    cuisines = list(set([item.lower().strip() for item in cuisines]))
    output["cuisine"] = ",".join(cuisines)
    output["delivery location"] = delivery_location
    output["is free parking"] = is_free_parking
    output["is dine-in"] = is_dine_in
    output["price range"] = price_range
    return output


restaurant_data['dietary_preference'] = restaurant_data.apply(lambda x: dietary_preference_mapping(x[['categories','name','state','feature_dict','price range']].to_dict()),axis=1)

In [46]:
user_req_desc = (restaurant_data[['categories','name','state','feature_dict','price range']].iloc[41])
user_req_desc

categories      Mexican Restaurant,Restaurant,Carry-out food,T...
name                                                     Del Taco
state                                                  California
feature_dict                                                   {}
price range                                               average
Name: 41, dtype: object

In [47]:
display(dietary_preference_mapping(user_req_desc))

{'cuisine': 'restaurant,carry-out food,tex-mex restaurants,mexican restaurant,southwestern restaurants,mexican restaurants,restaurants,fast food restaurant,american restaurants',
 'delivery location': 'California',
 'is free parking': 'no',
 'is dine-in': 'yes',
 'price range': 'average'}

In [48]:
# # Data type of the output response
print(type(dietary_preference_mapping(user_req_desc)))

<class 'dict'>


### `compare_with_user_preferences()`:

This function compares the user's dietary preferences with the restaurants and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the restaurants based on their dietary preferences.
    - Calculate a score for each restaurant based on how well it matches the user's requirements.
    - Sort the restaurants based on their scores in descending order.
    - Return the top restaurants as a JSON-formatted string.

In [49]:

def compare_with_user_preferences(df,response_dict_n):
    user_requirements = response_dict_n
    location = user_requirements['delivery location']
    # Extracting user requirements from the input string (assuming it's a dictionary)
    # Since the function parameter already seems to be a string, we'll use it directly instead of extracting from a dictionary

    filtered_df = df.loc[(df["state"]==location) | (df["province"]==location)].reset_index(drop=True)
    # # # Creating a new column 'score' in the filtered DataFrame and initializing it to 0
    filtered_df['score'] = 0
    # # # Iterating over each restaurants in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_df.iterrows():
        user_product_match = row['dietary_preference']
        score = 0
        # Comparing user dietary preferences with restaurants and update scores
        for key, user_value in user_requirements.items():
            if key in (['is free parking','is dine-in','drive through']):
                mappings = {'no': 0, 'yes': 1}
                user_value = user_requirements.get(key, None)
                rest_value = user_product_match.get(key, None)
                rest_mapping = mappings.get(rest_value, -1)
                user_mapping = mappings.get(user_value, -1)
                if rest_mapping >= user_mapping:
                    score += 1
            elif key == 'price range':
                mappings = {'below average': 0, 'average': 1, 'above average':2}
                user_value = user_requirements.get(key, None)
                rest_value = user_product_match.get(key, None)
                rest_mapping = mappings.get(rest_value, -1)
                user_mapping = mappings.get(user_value, -1)
                if rest_mapping >= user_mapping:
                    score += 1
            else:
                rest_data = user_product_match.get(key, None)
                if rest_data and any(item.strip() in rest_data.split(',') for item in user_value.split(',')):
                    score += 1  # Incrementing score if restaurant value meets or exceeds user value

            filtered_df.at[index, 'score'] = score 


    # Sorting restaurant by score in descending order 
    top_items = filtered_df.sort_values('score', ascending=False)[['categories','name','state','feature_dict','price range','score']].head(5)
    top_items_json = top_items.to_json(orient='records')  # Converting the top restaurant DataFrame to JSON format

    # top_restaurants
    return top_items_json

Now that you have the `compare_with_user_preferences()` function ready, let's pass the `response_dict_n` to the function to get top restaurant recommendations.

In [50]:
response_dict_n

{'cuisine': 'burger',
 'delivery location': 'TX',
 'is free parking': 'No',
 'is dine-in': 'Yes',
 'price range': 'average'}

In [51]:
display(response_dict_n, '\n',type(response_dict_n))

{'cuisine': 'burger',
 'delivery location': 'TX',
 'is free parking': 'No',
 'is dine-in': 'Yes',
 'price range': 'average'}

'\n'

dict

In [52]:
dictionary_present(response_dict_n)

{'cuisine': 'burger',
 'delivery location': 'TX',
 'is free parking': 'No',
 'is dine-in': 'Yes',
 'price range': 'average'}

In [53]:
# restaurant_data['dietary_preference'] = restaurant_data.apply(lambda x: product_map_layer(str(x[['categories','name','state','feature_dict','price range']].to_dict())),axis=1)

In [54]:
top_restaurants = compare_with_user_preferences(restaurant_data,response_dict_n)

In [55]:
# Get output in JSON Format
top_restaurants_json = json.loads(top_restaurants)
# type(top_restaurants_json)
top_restaurants_json

[{'categories': 'Carry-out food,Delicatessens,Take Out Restaurants,Fast Food Restaurants,Sandwiches,Restaurants,Sandwich Shops',
  'name': 'SUBWAY',
  'state': 'Texas',
  'feature_dict': {},
  'price range': 'average',
  'score': 3},
 {'categories': 'Fried Chicken Joint,Fast Food Restaurants,Take Out Restaurants,Chicken Restaurants,Catering,Restaurants',
  'name': 'Kfc',
  'state': 'Texas',
  'feature_dict': {},
  'price range': 'average',
  'score': 3},
 {'categories': 'Carry-out food,Delicatessens,Take Out Restaurants,Fast Food Restaurants,Sandwiches,Restaurants,Sandwich Shops',
  'name': 'SUBWAY',
  'state': 'Texas',
  'feature_dict': {},
  'price range': 'average',
  'score': 3},
 {'categories': 'Delis And Delicatessens,Restaurant,Take Out Restaurants,Fast Food Restaurants,Restaurants,Sandwich Shops',
  'name': 'Subway',
  'state': 'Texas',
  'feature_dict': {'open-hours': 'regular hoursmon - sun:9:00 am - 10:00 pm',
   'payment': 'master card, amex, visa, discover, company card, d

In [56]:
def recommendation_validation(recommendation):
    data = json.loads(recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['score'] > 2:
            data1.append(data[i])

    return data1

In [57]:
validated_data = recommendation_validation(top_restaurants)
display(validated_data,'\n')

[{'categories': 'Carry-out food,Delicatessens,Take Out Restaurants,Fast Food Restaurants,Sandwiches,Restaurants,Sandwich Shops',
  'name': 'SUBWAY',
  'state': 'Texas',
  'feature_dict': {},
  'price range': 'average',
  'score': 3},
 {'categories': 'Fried Chicken Joint,Fast Food Restaurants,Take Out Restaurants,Chicken Restaurants,Catering,Restaurants',
  'name': 'Kfc',
  'state': 'Texas',
  'feature_dict': {},
  'price range': 'average',
  'score': 3},
 {'categories': 'Carry-out food,Delicatessens,Take Out Restaurants,Fast Food Restaurants,Sandwiches,Restaurants,Sandwich Shops',
  'name': 'SUBWAY',
  'state': 'Texas',
  'feature_dict': {},
  'price range': 'average',
  'score': 3},
 {'categories': 'Delis And Delicatessens,Restaurant,Take Out Restaurants,Fast Food Restaurants,Restaurants,Sandwich Shops',
  'name': 'Subway',
  'state': 'Texas',
  'feature_dict': {'open-hours': 'regular hoursmon - sun:9:00 am - 10:00 pm',
   'payment': 'master card, amex, visa, discover, company card, d

'\n'

Now that you the top restaurants extracted, let's pass it to the recommendation layer that'll send it to the user and the user can ask questions around it.

## Stage 3

### Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_with_user_preferences` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. Initialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.



In [58]:
def initialize_conv_reco(recommendations):
    system_message = f"""
    You are an excellent food blogger or food critic expert and your goal is to recommend the best restaurant for a user in \
    'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'. \

    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each restaurant, the food they are offering in the following format:
    1. <Restaurant Name> : <location>, <Cuisines offered by them>
    2. <Restaurant Name> : <location>, <Cuisines offered by them>

    """
    user_message = f""" These are the recommendations: {recommendations}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

Let's initialize the conversation for recommendation.

In [59]:
debug_conversation_reco = initialize_conv_reco(top_restaurants)
debug_conversation_reco

[{'role': 'system',
  'content': "\n    You are an excellent food blogger or food critic expert and your goal is to recommend the best restaurant for a user in     'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'. \n    You should keep the user profile in mind while answering the questions.\n    Start with a brief summary of each restaurant, the food they are offering in the following format:\n    1. <Restaurant Name> : <location>, <Cuisines offered by them>\n    2. <Restaurant Name> : <location>, <Cuisines offered by them>\n\n    "},
 {'role': 'user',
  'content': ' These are the recommendations: [{"categories":"Carry-out food,Delicatessens,Take Out Restaurants,Fast Food Restaurants,Sandwiches,Restaurants,Sandwich Shops","name":"SUBWAY","state":"Texas","feature_dict":{},"price range":"average","score":3},{"categories":"Fried Chicken Joint,Fast Food Restaurants,Take Out Restaurants,Chicken Restaurants,Catering,Restaurants","name":"Kfc","state":"Texas","featu

Let's see what the assistant responds with the new initialization.

In [60]:
debug_recommendation = get_chat_completions(debug_conversation_reco)
print(debug_recommendation + '\n')

1. SUBWAY : Texas, Sandwiches
2. KFC : Texas, Fried Chicken, Fast Food

Based on the recommendations provided in Texas, I would suggest trying out KFC for some delicious fried chicken and fast food experience. If you're in the mood for sandwiches, SUBWAY is a good option as well. Both restaurants offer a casual dining experience suitable for a quick meal on the go.



Now, you can converse with the chatbot on the filtered products.

In [61]:
debug_conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response_dict_n)})
debug_conversation_reco.append({"role": "assistant", "content": debug_recommendation})

In [62]:
debug_conversation_reco.append({"role": "user", "content": debug_user_input})
debug_response_asst_reco = get_chat_completions(debug_conversation_reco)
display('\n' + debug_response_asst_reco + '\n')

'\n1. SUBWAY : Texas, Sandwiches\n2. KFC : Texas, Fried Chicken, Fast Food\n\nHi Krishna! Based on your user profile and the recommendations available in Texas, I would recommend trying out KFC for a fulfilling dinner experience. Enjoy some delicious fried chicken and fast food options at KFC. If you prefer sandwiches, SUBWAY is also a good choice for a quick and satisfying meal. Both restaurants offer dine-in options and fall within an average price range. Have a great dinner!\n'

### Dialogue Management System

Bringing everything together, we create a `diagloue_mgmt_system()` function that contains the logic of how the different layers would interact with each other. This will be the function that we'll call to initiate the chatbot

In [77]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    print(introduction)

    top_restaurants = None

    user_input = ''

    while(user_input.lower() != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_restaurants is None:

            conversation.append({"role": "user", "content": user_input})
            response_assistant = get_chat_completions(conversation)
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)
#             print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
#                 print('\n' + "Variables extracted!" + '\n')
                response = dictionary_present(response_assistant)

                print("Thank you for providing all the information. Kindly wait, while I recommend restaurants: \n")
                top_restaurants = compare_with_user_preferences(restaurant_data,response_dict_n)

                validated_reco = recommendation_validation(top_restaurants)

                conversation_reco = initialize_conv_reco(validated_reco)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n')
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})

Execute **dialogue_mgmt_system()** to run the restaurant recommendation assist chatbot

In [78]:
dialogue_mgmt_system()

Welcome to the restaurant recommendation chat! I'm here to help you find the perfect place to dine. Please let me know your ideal location from the following states in the US - 'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' or 'Washington'. You can type 'Exit' to end the chat at any time. So, where would you like to dine today?
florida

Welcome! I can help you find the perfect restaurant in Florida. 
What type of cuisine are you in the mood for today?

chinese

Great choice! Do you prefer dining in or takeout for your meal today?

yes

I'm sorry, I should have been clearer. Do you prefer to dine in at the restaurant or would you like to order takeout for your meal?

dine in

Would you prefer a restaurant with free parking facilities for your dine-in experience or is that not a requirement for you?

yes

Thank you for clarifying. And lastly, could you please let me know your budget for this meal? This will help me recommend restaurants that fit within your price range.

10

## Future Scope of Work and Conclusion

### Future Scope of Work

1. Now it is limited to a few states 'New York', 'Florida', 'California', 'Pennsylvania', 'Texas' and 'Washington’ in United States due to limited availability of data and time constraints for further web scraping. Need to extend this to the remaining states and gradually global wise.
2. Integration with user interface (front end web) for user authentication and sharing recommendations.
3. Incorporation of machine learning models for real-time personalization and recommendation refinement.



###  Conclusion:

The Restaurant Recommendation System aims to provide users with personalized recommendations based on their preferences, location, budget, and dietary restrictions. By leveraging data from multiple sources and implementing advanced recommendation algorithms, the system aims to enhance the dining experience for users and help them discover new dining options quickly and conveniently.

